### Load data

In [1]:
# Import packages
import pandas as pd
import joblib
import os
import warnings

from tqdm import tqdm
from pymatgen import MPRester

In [2]:
# Import custom packages
import sys
sys.path.insert(1, 'C:/Users/Songy/OneDrive/Documents/GBSFS4MPP_local/new_properties')
warnings.filterwarnings("ignore")

import create_material_objects as cmo

In [3]:
# Load dataset
df = joblib.load('ricci_boltztrap_mp_tabular.pkl') 
df.head()

,task,functional,is_metal,ΔE [eV],V [Å³],S.p [µV/K],S.n [µV/K],Sᵉ.p.v [µV/K],Sᵉ.p.T [K],Sᵉ.p.c [cm⁻³],...,mₑᶜ.p.ε̄ [mₑ],mₑᶜ.p.ε₁ [mₑ],mₑᶜ.p.ε₂ [mₑ],mₑᶜ.p.ε₃ [mₑ],mₑᶜ.n.ε̄ [mₑ],mₑᶜ.n.ε₁ [mₑ],mₑᶜ.n.ε₂ [mₑ],mₑᶜ.n.ε₃ [mₑ],structure,pretty_formula
mp-623511,mp-942243,GGA,Yes,0.000000,28.290636,1.291511,1.292671,0.082688,100.0,1.000000e+20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[[0. 0. 0.] Sn],Sn
mp-1992,mp-942015,GGA,Yes,0.000000,332.023436,2.819687,2.820492,17.617166,1300.0,1.000000e+20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[[0.95193129 2.36031189 2.83455083] Pu, [ 4.45...",Pu5Ru3
mp-27502,mp-670711,GGA,No,5.835400,144.673148,770.127260,480.064260,1329.539967,1300.0,1.000000e+16,...,9.948649,3.481600,5.835956,20.528390,0.990398,0.581862,1.084788,1.304544,"[[0. 2.44550604 0. ] C, [2.1193...",CNCl
mp-770629,mp-811645,GGA+U,No,3.092696,350.609125,747.876160,558.917437,1282.634267,1100.0,1.000000e+16,...,10.016446,7.157813,8.633919,14.257605,3.746585,1.077907,4.539832,5.622017,[[9.12865390e-01 1.58244593e-03 6.52361417e+00...,Li2AlFeO4
mp-27202,mp-860107,GGA,No,4.156906,841.505130,752.459840,777.549330,1278.365767,1000.0,1.000000e+16,...,25.742400,17.822069,20.141565,39.263564,19.824545,15.836995,18.160493,25.476146,"[[6.85787739 5.82686049 2.92762597] B, [1.8867...",B9H8S


In [4]:
# Get all the mp-id with NaN
df = df[['structure', 'pretty_formula']]
df_na = df[df.isna().any(axis=1)]
df_na


,structure,pretty_formula
mp-769834,NaN,NaN
mp-764217,NaN,NaN
mp-25058,NaN,NaN
mp-764864,NaN,NaN
mp-764165,NaN,NaN
...,...,...
mp-565480,NaN,NaN
mp-773740,NaN,NaN
mp-25604,NaN,NaN
mp-764227,NaN,NaN


In [5]:
# Get structure info for missing data by 
MP_API_KEY=os.environ.get("qfqsDIFq4Fef4Zvm26DQ")

with MPRester('qfqsDIFq4Fef4Zvm26DQ') as mpr:
    for i in tqdm(range(len(df_na))):
        structure = mpr.get_structure_by_material_id(df_na.index[i])
        
        # Store structure info
        df_na.iloc[i, :][0] = structure
        
        # Pretty formula
        df_na.iloc[i, :][1] = structure.composition.reduced_formula
        
        # structure = mpr.get_structure_by_material_id("mp-1992")
        # structure = mpr.get_task_data("mp-769834")

df_na

100%|██████████| 10347/10347 [2:01:26<00:00,  1.42it/s] 


,structure,pretty_formula
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6
...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4


In [28]:
# with MPRester('qfqsDIFq4Fef4Zvm26DQ') as mpr:
#     for i in range(len(df_na)):
#         structure = mpr.get_materials_id_from_task_id('mp-769834')
#         print(structure)

# structure.composition.reduced_formula
# structure.composition.formula

In [6]:
# Save data
# joblib.dump(df_na, 'ricci_boltztrap_mp_tabular_with_NaN.pkl')

['ricci_boltztrap_mp_tabular_with_NaN.pkl']

In [9]:
a = joblib.load('ricci_boltztrap_mp_tabular_with_NaN.pkl')
a

,structure,pretty_formula
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6
...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4


In [10]:
a[a.isna().any(axis=1)]

,structure,pretty_formula


In [14]:
# Material objects
df_na
df = df_na[df_na['pretty_formula'].notna()]
df

,structure,pretty_formula
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6
...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4


In [15]:
# Initialize to create/convert material object (pymatgen)
converter = cmo.material_object(df=df)

# Possible functions
# Create Composition object from chemical formula
# df = converter.formula2composition(formula_col='formula')

# Create Composition object from Structure object
# df = converter.structure2composition(structure_col='structure')

# Create Oxidation Structure object from Structure object
# df = converter.structure2oxidstructure(structure_col='structure')

# Create Oxidation Composition object from Composition object
# df = converter.composition2oxidcomposition(composition_col='composition')

# Source Structure Object from MP database using Composition object 
# df = converter.composition2structure_fromMP(composition_col='composition', mapi_key='lZCh9ke4qRxMQO16')

In [16]:
# Create Composition object from chemical formula
df = converter.structure2composition(structure_col='structure')
df

StructureToComposition:   0%|          | 0/10347 [00:00<?, ?it/s]

,structure,pretty_formula,composition
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)"
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)"
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)"
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)"
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)"
...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)"
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)"
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)"
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)"


In [8]:
# # Optional - Create Oxidation Structure object from Structure object
# # Create Oxidation Composition object from Composition object
# df = converter.composition2oxidcomposition(composition_col='composition')
# df

CompositionToOxidComposition:   0%|          | 0/37390 [00:00<?, ?it/s]

In [ ]:
# # Optional - Create Oxidation Structure object from Structure object
# # Needed for generate_oxid_structural_features function, which has the ElectronicRadialDistributionFunction

# df = converter.structure2oxidstructure(structure_col='structure') 
# df

In [17]:
joblib.dump(df, 'ricci_boltztrap_mp_tabular_material_objects_witih_NaN.pkl')

['ricci_boltztrap_mp_tabular_material_objects_witih_NaN.pkl']

#### Generate Features

##### (1) Composition

In [1]:
# Import packages
import pandas as pd
import joblib

In [2]:
# Import custom packages
import sys
sys.path.insert(1, 'C:/Users/Songy/OneDrive/Documents/GBSFS4MPP_local/new_properties')

import featurizers as feat

In [3]:
# Load Materials object file
df_obj = joblib.load('ricci_boltztrap_mp_tabular_material_objects_witih_NaN.pkl')
df_obj

,structure,pretty_formula,composition
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)"
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)"
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)"
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)"
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)"
...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)"
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)"
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)"
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)"


In [4]:
# Initialise featurizers script
descriptors = feat.prepare_to_featurize(df=df_obj)

In [5]:
# Features based on oxidation composition
df_com = descriptors.generate_oxid_composition_features_with_composition()
df_com

# # Features based on oxidation composition
# df_com = descriptors.generate_oxid_composition_features()
# df_com

MultipleFeaturizer:   0%|          | 0/10347 [00:00<?, ?it/s]

Total no. of features generated: 13


,structure,pretty_formula,composition,minimum oxidation state,maximum oxidation state,range oxidation state,std_dev oxidation state,compound possible,max ionic char,avg ionic char,avg anion electron affinity,minimum EN difference,maximum EN difference,range EN difference,mean EN difference,std_dev EN difference
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)",0.0,0.0,0.0,0.0,True,0.714753,0.152028,0.0,0.0,0.0,0.0,0.0,0.0
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)",0.0,0.0,0.0,0.0,False,0.894601,0.196985,0.0,0.0,0.0,0.0,0.0,0.0
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)",0.0,0.0,0.0,0.0,True,0.820234,0.113492,0.0,0.0,0.0,0.0,0.0,0.0
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)",0.0,0.0,0.0,0.0,True,0.779730,0.131521,0.0,0.0,0.0,0.0,0.0,0.0
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)",0.0,0.0,0.0,0.0,True,0.779730,0.169201,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)",0.0,0.0,0.0,0.0,True,0.820234,0.136348,0.0,0.0,0.0,0.0,0.0,0.0
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)",0.0,0.0,0.0,0.0,True,0.685139,0.140569,0.0,0.0,0.0,0.0,0.0,0.0
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)",0.0,0.0,0.0,0.0,False,0.443019,0.085693,0.0,0.0,0.0,0.0,0.0,0.0
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)",0.0,0.0,0.0,0.0,True,0.774266,0.185501,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Features based on composition
df_com = descriptors.generate_composition_features()
df_com

MultipleFeaturizer:   0%|          | 0/10347 [00:00<?, ?it/s]

Total no. of features generated: 610


,structure,pretty_formula,composition,minimum oxidation state,maximum oxidation state,range oxidation state,std_dev oxidation state,compound possible,max ionic char,avg ionic char,...,frac p valence electrons,frac d valence electrons,frac d valence electrons,frac f valence electrons,frac f valence electrons,mean simul. packing efficiency,mean abs simul. packing efficiency,dist from 1 clusters |APE| < 0.010,dist from 3 clusters |APE| < 0.010,dist from 5 clusters |APE| < 0.010
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)",0.0,0.0,0.0,0.0,True,0.714753,0.152028,...,0.378378,0.135135,0.135135,0.189189,0.189189,0.049020,0.049020,0.037675,0.059724,0.079150
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)",0.0,0.0,0.0,0.0,False,0.894601,0.196985,...,0.512195,0.073171,0.073171,0.000000,0.000000,0.000000,0.000000,0.107485,0.109974,0.115588
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)",0.0,0.0,0.0,0.0,True,0.820234,0.113492,...,0.576271,0.084746,0.084746,0.000000,0.000000,0.008726,0.052126,0.018539,0.048960,0.055528
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)",0.0,0.0,0.0,0.0,True,0.779730,0.131521,...,0.440678,0.203390,0.203390,0.000000,0.000000,0.025481,0.026335,0.025242,0.051269,0.057416
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)",0.0,0.0,0.0,0.0,True,0.779730,0.169201,...,0.428571,0.214286,0.214286,0.000000,0.000000,0.022375,0.027771,0.025242,0.035643,0.039095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)",0.0,0.0,0.0,0.0,True,0.820234,0.136348,...,0.514286,0.152381,0.152381,0.000000,0.000000,0.072347,0.074382,0.000000,0.031603,0.041558
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)",0.0,0.0,0.0,0.0,True,0.685139,0.140569,...,0.433071,0.283465,0.283465,0.000000,0.000000,0.000000,0.000000,0.476294,0.498931,0.521431
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)",0.0,0.0,0.0,0.0,False,0.443019,0.085693,...,0.487179,0.205128,0.205128,0.000000,0.000000,-0.001031,0.008243,0.000000,0.030889,0.048244
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)",0.0,0.0,0.0,0.0,True,0.774266,0.185501,...,0.432432,0.135135,0.135135,0.000000,0.000000,0.004911,0.007805,0.043012,0.052388,0.066788


In [7]:
joblib.dump(df_com, 'ricci_boltztrap_mp_tabular_composition_with_NaN.pkl')

['ricci_boltztrap_mp_tabular_composition_with_NaN.pkl']

##### (2) Structural - Section skipped

In [1]:
# Import packages
import pandas as pd
import joblib

In [2]:
# Import custom packages
import sys
sys.path.insert(1, 'C:/Users/Songy/OneDrive/Documents/GBSFS4MPP_local/new_properties')

#import featurizers as feat
import featurizers_light as feat

In [3]:
# Load Materials object file
df_obj = joblib.load('ricci_boltztrap_mp_tabular_material_objects_witih_NaN.pkl')
df_obj

,structure,pretty_formula,composition
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)"
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)"
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)"
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)"
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)"
...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)"
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)"
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)"
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)"


In [4]:
# Initialization of featurizer
descriptors = feat.prepare_to_featurize(df=df_obj)

In [12]:
# # Features based on oxidation structure
# df_struc = descriptors.generate_oxid_structural_features()
# df_struc

In [13]:
# Features based on structure
df_struc = descriptors.generate_structural_features()
df_struc

MultipleFeaturizer:   0%|          | 0/10347 [00:00<?, ?it/s]

Total no. of features generated: 123


,structure,pretty_formula,composition,density,vpa,packing fraction,mean ordering parameter shell 1,mean ordering parameter shell 2,mean ordering parameter shell 3,max packing efficiency,...,maximum local difference in SpaceGroupNumber,range local difference in SpaceGroupNumber,mean local difference in SpaceGroupNumber,avg_dev local difference in SpaceGroupNumber,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,n_symmetry_ops,dimensionality
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)",6.657757,11.921997,0.628062,0.568049,0.160359,0.106801,0.366327,...,217.000000,117.668216,143.296521,49.238893,227,cubic,1,True,4,3
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)",2.442965,10.420925,0.683057,0.720897,0.492683,0.245278,0.338039,...,214.131917,63.668625,180.861123,23.752262,4,monoclinic,6,False,2,3
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)",3.125181,14.924983,0.439064,0.606491,0.329275,0.219262,0.205709,...,217.000000,206.991306,81.558817,50.678839,14,monoclinic,6,True,4,3
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)",3.241676,11.001643,0.382849,0.740495,0.413658,0.197745,0.186106,...,211.766133,102.612994,157.145897,27.566106,1,triclinic,7,False,1,3
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)",3.650266,11.067586,0.537613,0.699746,0.465127,0.236280,0.346460,...,199.486898,60.295565,170.218412,21.259217,14,monoclinic,6,True,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)",3.456222,14.926922,0.560189,0.781788,0.350449,0.233698,0.226080,...,216.976821,206.976821,111.929929,53.780009,14,monoclinic,6,True,4,3
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)",4.305102,11.744053,0.364976,0.359510,0.097414,0.059496,0.340197,...,215.693477,137.736125,131.672455,55.785684,1,triclinic,7,False,1,3
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)",4.137692,10.278127,0.293729,0.426493,0.227213,0.103437,0.214437,...,213.000000,203.023043,67.447072,48.517643,63,orthorhombic,5,True,4,3
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)",3.695723,13.432774,0.822617,0.968154,0.921749,0.883886,0.513197,...,209.834611,5.352626,207.402464,1.832630,65,orthorhombic,5,True,4,3


In [14]:
joblib.dump(df_struc, 'ricci_boltztrap_mp_tabular_structural_with_NaN.pkl')

['ricci_boltztrap_mp_tabular_structural_with_NaN.pkl']

In [5]:
# Initialization of featurizer
descriptors = feat.prepare_to_featurize(df=df_obj)

# Features based on JarvisCFID
df_Jarvis = descriptors.generate_JarvisCFID_features()

MultipleFeaturizer:   0%|          | 0/10347 [00:00<?, ?it/s]

Total no. of features generated: 1557


In [6]:
joblib.dump(df_Jarvis, 'ricci_boltztrap_mp_tabular_structural_jarviscfid_with_NaN.pkl')

['ricci_boltztrap_mp_tabular_structural_jarviscfid_with_NaN.pkl']

In [12]:
df_Jarvis

,structure,pretty_formula,composition,jml_bp_mult_atom_rad,jml_hfus_add_bp,jml_elec_aff_mult_voro_coord,jml_mol_vol_subs_atom_mass,jml_is_halogen,jml_atom_mass_subs_first_ion_en,jml_row,...,jml_nn_91,jml_nn_92,jml_nn_93,jml_nn_94,jml_nn_95,jml_nn_96,jml_nn_97,jml_nn_98,jml_nn_99,jml_nn_100
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)",0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)",1851.596250,1367.619937,6.68250,-10.745076,0.250000,12.425386,2.500000,...,11.272727,8.000000,10.000000,9.454545,9.636364,12.000000,11.272727,8.727273,13.090909,11.090909
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)",2499.230000,1646.196437,4.96500,-23.077865,0.000000,36.618407,3.500000,...,8.272727,6.818182,7.636364,6.181818,6.545455,7.909091,10.454545,9.818182,8.454545,5.545455
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)",2588.367500,2259.968075,3.27375,-11.934100,0.000000,13.596553,2.500000,...,1.090909,3.227273,1.590909,1.295455,1.272727,1.818182,1.295455,1.113636,1.363636,0.818182
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)",2661.867500,1899.875700,2.71850,-19.343861,0.000000,23.118384,3.000000,...,10.484848,12.848485,10.060606,11.272727,11.818182,10.303030,8.424242,10.545455,12.242424,9.515152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)",2430.094000,1696.352100,6.35760,-28.271692,0.000000,40.834805,3.600000,...,6.631579,6.736842,7.789474,9.789474,9.263158,6.526316,9.789474,6.736842,8.526316,8.947368
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)",1494.745000,1103.124283,5.76000,-18.397601,0.333333,17.299851,2.666667,...,6.333333,10.888889,10.111111,11.666667,12.111111,12.222222,11.333333,6.555556,7.555556,8.222222
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)",1635.073333,1275.520800,7.09000,-21.565521,0.000000,24.640674,3.000000,...,6.000000,8.666667,12.333333,9.333333,7.333333,11.000000,14.666667,7.000000,9.333333,14.666667
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)",2161.440000,1393.805567,1.50480,-20.035148,0.000000,27.950072,3.333333,...,5.625000,5.875000,0.562500,8.875000,8.000000,5.500000,9.125000,10.875000,9.625000,4.062500


##### (3) Extra Features

In [7]:
# Import packages
import pandas as pd
import joblib

from pymatgen import core

In [8]:
# Import custom packages
import sys
sys.path.insert(1, 'C:/Users/Songy/OneDrive/Documents/GBSFS4MPP_local/new_properties')

import featurizers as feat

In [9]:
# Load Materials object file
df_obj = joblib.load('ricci_boltztrap_mp_tabular_material_objects_witih_NaN.pkl')
df_obj

,structure,pretty_formula,composition
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)"
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)"
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)"
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)"
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)"
...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)"
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)"
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)"
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)"


In [10]:
# Initialization of featurizer
descriptors = feat.prepare_to_featurize(df=df_obj)

df_custom = descriptors.custom_features(composition_col_exist=True)
df_custom

,structure,pretty_formula,composition,weight,total_e,avg_electroneg,noble_gas,transition_metal,post_transition_metal,rare_earth_metal,...,alkaline,halogen,chalcogen,lanthanoid,actinoid,quadrupolar,s-block,p-block,d-block,f-block
mp-769834,"[[3.81044025 1.27014675 3.81044025] Eu, [3.810...",Eu2Mn2O7,"(Eu, Mn, O)",1051.599780,464.0,2.689091,0,1,0,1,...,0,0,1,1,0,1,0,1,1,1
mp-764217,"[[0.59367752 5.88084745 4.81026433] Li, [3.930...",Li5VO4F,"(Li, V, O, F)",337.285006,158.0,2.206364,0,1,0,0,...,0,1,1,0,0,1,1,1,1,0
mp-25058,"[[1.82683783 3.70050826 7.67322499] K, [3.4877...",KMoP2O7,"(K, Mo, P, O)",1235.926496,588.0,2.858182,0,1,0,0,...,0,0,1,0,0,1,1,1,1,0
mp-764864,"[[-6.17916457 10.61753344 -4.20923459] Li, [-6...",LiFe2(BO3)2,"(Li, Fe, B, O)",944.997600,452.0,2.669091,0,1,0,0,...,0,0,1,0,0,1,1,1,1,0
mp-764165,"[[2.87312411 7.42944559 3.55127513] Li, [2.405...",Li2TiMn2O6,"(Li, Ti, Mn, O)",1605.728940,756.0,2.476364,0,1,0,0,...,0,0,1,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-565480,"[[4.24379866 2.90151778 1.01710537] Rb, [7.393...",Rb2V2Cu(PO6)2,"(Rb, V, Cu, P, O)",1180.609848,550.0,2.761053,0,1,0,0,...,0,0,1,0,0,1,1,1,1,0
mp-773740,"[[2.83417266 5.14658129 5.14898174] Fe, [2.505...",Fe6O5F7,"(Fe, O, F)",548.055822,259.0,3.113333,0,1,0,0,...,0,1,1,0,0,1,0,1,1,0
mp-25604,"[[0. 0. 3.08825] Ni, [0. 0. 0.] Ni, ...",NiPO4,"(Ni, P, O)",307.329524,150.0,2.976667,0,1,0,0,...,0,0,1,0,0,1,0,1,1,0
mp-764227,"[[-4.752133 -9.51075463 -0.02437138] Ca, [-4...",Ca3MnO4,"(Ca, Mn, O)",956.678580,468.0,2.288750,0,1,0,0,...,1,0,1,0,0,1,1,1,1,0


In [11]:
joblib.dump(df_custom, 'ricci_boltztrap_mp_tabular_additional_features_with_NaN.pkl')

['ricci_boltztrap_mp_tabular_additional_features_with_NaN.pkl']

#### Merge results

In [1]:
# Import packages
import pandas as pd
import joblib
import re

from pymatgen import core

In [2]:
# Import custom packages
import sys
sys.path.insert(1, 'C:/Users/Songy/OneDrive/Documents/GBSFS4MPP_local/new_properties')

import featurizers as feat

In [3]:
# Load Materials object file
cols = ['structure', 'pretty_formula', 'composition']

df1 = joblib.load('ricci_boltztrap_mp_tabular_composition_with_NaN.pkl')
df1 = df1.drop(cols, axis=1)

df2 = joblib.load('ricci_boltztrap_mp_tabular_structural_with_NaN.pkl')
df2 = df2.drop(cols, axis=1)

df3 = joblib.load('ricci_boltztrap_mp_tabular_structural_jarviscfid_with_NaN.pkl')
df3 = df3.drop(cols, axis=1)

df4 = joblib.load('ricci_boltztrap_mp_tabular_additional_features_with_NaN.pkl')
df4 = df4.drop(cols, axis=1)

In [4]:
# Merge on index
df_merged = pd.merge(df1, df2, left_index=True, right_index=True)
df_merged = pd.merge(df_merged, df3, left_index=True, right_index=True)
df_merged = pd.merge(df_merged, df4, left_index=True, right_index=True)
df_merged

,minimum oxidation state,maximum oxidation state,range oxidation state,std_dev oxidation state,compound possible,max ionic char,avg ionic char,avg anion electron affinity,minimum EN difference,maximum EN difference,...,alkaline,halogen,chalcogen,lanthanoid,actinoid,quadrupolar,s-block,p-block,d-block,f-block
mp-769834,0.0,0.0,0.0,0.0,True,0.714753,0.152028,0.0,0.0,0.0,...,0,0,1,1,0,1,0,1,1,1
mp-764217,0.0,0.0,0.0,0.0,False,0.894601,0.196985,0.0,0.0,0.0,...,0,1,1,0,0,1,1,1,1,0
mp-25058,0.0,0.0,0.0,0.0,True,0.820234,0.113492,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,0
mp-764864,0.0,0.0,0.0,0.0,True,0.779730,0.131521,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,0
mp-764165,0.0,0.0,0.0,0.0,True,0.779730,0.169201,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-565480,0.0,0.0,0.0,0.0,True,0.820234,0.136348,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,0
mp-773740,0.0,0.0,0.0,0.0,True,0.685139,0.140569,0.0,0.0,0.0,...,0,1,1,0,0,1,0,1,1,0
mp-25604,0.0,0.0,0.0,0.0,False,0.443019,0.085693,0.0,0.0,0.0,...,0,0,1,0,0,1,0,1,1,0
mp-764227,0.0,0.0,0.0,0.0,True,0.774266,0.185501,0.0,0.0,0.0,...,1,0,1,0,0,1,1,1,1,0


In [5]:
# Remove duplications
df_merged  = df_merged.loc[:,~df_merged.columns.duplicated()]

# Remove space and JSON character in the column
df_merged.columns = df_merged.columns.str.replace(' ','_')
df_merged = df_merged.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

# False and True -> 0 and 1
df_merged.replace({False: 0, True: 1}, inplace=True)
df_merged

,minimum_oxidation_state,maximum_oxidation_state,range_oxidation_state,std_dev_oxidation_state,compound_possible,max_ionic_char,avg_ionic_char,avg_anion_electron_affinity,minimum_EN_difference,maximum_EN_difference,...,alkaline,halogen,chalcogen,lanthanoid,actinoid,quadrupolar,sblock,pblock,dblock,fblock
mp-769834,0.0,0.0,0.0,0.0,1,0.714753,0.152028,0.0,0.0,0.0,...,0,0,1,1,0,1,0,1,1,1
mp-764217,0.0,0.0,0.0,0.0,0,0.894601,0.196985,0.0,0.0,0.0,...,0,1,1,0,0,1,1,1,1,0
mp-25058,0.0,0.0,0.0,0.0,1,0.820234,0.113492,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,0
mp-764864,0.0,0.0,0.0,0.0,1,0.779730,0.131521,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,0
mp-764165,0.0,0.0,0.0,0.0,1,0.779730,0.169201,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-565480,0.0,0.0,0.0,0.0,1,0.820234,0.136348,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,0
mp-773740,0.0,0.0,0.0,0.0,1,0.685139,0.140569,0.0,0.0,0.0,...,0,1,1,0,0,1,0,1,1,0
mp-25604,0.0,0.0,0.0,0.0,0,0.443019,0.085693,0.0,0.0,0.0,...,0,0,1,0,0,1,0,1,1,0
mp-764227,0.0,0.0,0.0,0.0,1,0.774266,0.185501,0.0,0.0,0.0,...,1,0,1,0,0,1,1,1,1,0


In [7]:
joblib.dump(df_merged, 'ricci_boltztrap_mp_tabular_merged_with_NaN_pre_ohe.pkl')

['ricci_boltztrap_mp_tabular_merged_with_NaN_pre_ohe.pkl']

In [31]:
# Initialization of featurizer
descriptors = feat.prepare_to_featurize(df=df_merged)

# One hot encoding of categorical features
df_merged, categorical_feature = descriptors.OHE()

categorical_feature

No. of categorical features: 5


['HOMO_character_ohe_0',
 'HOMO_character_ohe_d',
 'HOMO_character_ohe_f',
 'HOMO_character_ohe_p',
 'HOMO_character_ohe_s',
 'HOMO_element_ohe_0',
 'HOMO_element_ohe_Ac',
 'HOMO_element_ohe_Ag',
 'HOMO_element_ohe_Al',
 'HOMO_element_ohe_As',
 'HOMO_element_ohe_Au',
 'HOMO_element_ohe_B',
 'HOMO_element_ohe_Ba',
 'HOMO_element_ohe_Bi',
 'HOMO_element_ohe_Br',
 'HOMO_element_ohe_C',
 'HOMO_element_ohe_Ca',
 'HOMO_element_ohe_Ce',
 'HOMO_element_ohe_Cl',
 'HOMO_element_ohe_Co',
 'HOMO_element_ohe_Cr',
 'HOMO_element_ohe_Cu',
 'HOMO_element_ohe_Dy',
 'HOMO_element_ohe_Er',
 'HOMO_element_ohe_Eu',
 'HOMO_element_ohe_F',
 'HOMO_element_ohe_Fe',
 'HOMO_element_ohe_Ga',
 'HOMO_element_ohe_Ge',
 'HOMO_element_ohe_H',
 'HOMO_element_ohe_Hg',
 'HOMO_element_ohe_Ho',
 'HOMO_element_ohe_I',
 'HOMO_element_ohe_In',
 'HOMO_element_ohe_Ir',
 'HOMO_element_ohe_Mn',
 'HOMO_element_ohe_Mo',
 'HOMO_element_ohe_N',
 'HOMO_element_ohe_Nb',
 'HOMO_element_ohe_Nd',
 'HOMO_element_ohe_Ni',
 'HOMO_element_ohe

In [32]:
# Original dataframe with target variables
df5 = joblib.load('ricci_boltztrap_mp_tabular.pkl')
df5 = df5[df5.isna().any(axis=1)]
df5
df5 = df5.drop(['task', 'functional', 'structure', 'pretty_formula'], axis=1)
print(len(df5.columns))

# Merge on index
df_merged = pd.merge(df_merged, df5, left_index=True, right_index=True)
df_merged.head()

43


,minimum_oxidation_state,maximum_oxidation_state,range_oxidation_state,std_dev_oxidation_state,compound_possible,max_ionic_char,avg_ionic_char,avg_anion_electron_affinity,minimum_EN_difference,maximum_EN_difference,...,κₑᵉ.n.T [K],κₑᵉ.n.c [cm⁻³],mₑᶜ.p.ε̄ [mₑ],mₑᶜ.p.ε₁ [mₑ],mₑᶜ.p.ε₂ [mₑ],mₑᶜ.p.ε₃ [mₑ],mₑᶜ.n.ε̄ [mₑ],mₑᶜ.n.ε₁ [mₑ],mₑᶜ.n.ε₂ [mₑ],mₑᶜ.n.ε₃ [mₑ]
mp-769834,0.0,0.0,0.0,0.0,1,0.714753,0.152028,0.0,0.0,0.0,...,100.0,1.000000e+16,0.001271,0.001271,0.001271,0.001271,0.001271,0.001271,0.001271,0.001271
mp-764217,0.0,0.0,0.0,0.0,0,0.894601,0.196985,0.0,0.0,0.0,...,600.0,1.000000e+16,13.477146,9.172227,10.664496,20.594714,20.343978,5.355413,10.932419,44.744102
mp-25058,0.0,0.0,0.0,0.0,1,0.820234,0.113492,0.0,0.0,0.0,...,1200.0,1.000000e+16,4.970205,3.003270,3.080827,8.826518,1.101847,0.885456,0.971100,1.448986
mp-764864,0.0,0.0,0.0,0.0,1,0.779730,0.131521,0.0,0.0,0.0,...,100.0,1.000000e+16,4623.412319,1064.288501,3807.133120,8998.815336,364.259663,36.799712,90.142238,965.837038
mp-764165,0.0,0.0,0.0,0.0,1,0.779730,0.169201,0.0,0.0,0.0,...,100.0,1.000000e+16,24.129105,8.791804,27.653713,35.941799,6.287117,3.366816,5.930837,9.563699


In [33]:
df_merged.columns[-43:].tolist()

['is_metal',
 'ΔE [eV]',
 'V [Å³]',
 'S.p [µV/K]',
 'S.n [µV/K]',
 'Sᵉ.p.v [µV/K]',
 'Sᵉ.p.T [K]',
 'Sᵉ.p.c [cm⁻³]',
 'Sᵉ.n.v [µV/K]',
 'Sᵉ.n.T [K]',
 'Sᵉ.n.c [cm⁻³]',
 'σ.p [1/Ω/m/s]',
 'σ.n [1/Ω/m/s]',
 'PF.p [µW/cm/K²/s]',
 'PF.n [µW/cm/K²/s]',
 'σᵉ.p.v [1/Ω/m/s]',
 'σᵉ.p.T [K]',
 'σᵉ.p.c [cm⁻³]',
 'σᵉ.n.v [1/Ω/m/s]',
 'σᵉ.n.T [K]',
 'σᵉ.n.c [cm⁻³]',
 'PFᵉ.p.v [µW/cm/K²/s]',
 'PFᵉ.p.T [K]',
 'PFᵉ.p.c [cm⁻³]',
 'PFᵉ.n.v [µW/cm/K²/s]',
 'PFᵉ.n.T [K]',
 'PFᵉ.n.c [cm⁻³]',
 'κₑ.p [W/K/m/s]',
 'κₑ.n [W/K/m/s]',
 'κₑᵉ.p.v [W/K/m/s]',
 'κₑᵉ.p.T [K]',
 'κₑᵉ.p.c [cm⁻³]',
 'κₑᵉ.n.v [W/K/m/s]',
 'κₑᵉ.n.T [K]',
 'κₑᵉ.n.c [cm⁻³]',
 'mₑᶜ.p.ε̄ [mₑ]',
 'mₑᶜ.p.ε₁ [mₑ]',
 'mₑᶜ.p.ε₂ [mₑ]',
 'mₑᶜ.p.ε₃ [mₑ]',
 'mₑᶜ.n.ε̄ [mₑ]',
 'mₑᶜ.n.ε₁ [mₑ]',
 'mₑᶜ.n.ε₂ [mₑ]',
 'mₑᶜ.n.ε₃ [mₑ]']

In [34]:
feature = df_merged.columns[:-43].tolist()
print(len(feature))

2463


In [35]:
joblib.dump(df_merged, 'ricci_boltztrap_mp_tabular_merged_with_NaN.pkl')
joblib.dump(feature, 'features_with_NaN.pkl')

['features_with_NaN.pkl']

Merge the all materials

In [37]:
df_v1 = joblib.load('ricci_boltztrap_mp_tabular_merged_metal.pkl')
df_v2 = joblib.load('ricci_boltztrap_mp_tabular_merged_with_NaN.pkl')

In [38]:
frames = [df_v1, df_v2]

result = pd.concat(frames)
result

,minimum_oxidation_state,maximum_oxidation_state,range_oxidation_state,std_dev_oxidation_state,compound_possible,max_ionic_char,avg_ionic_char,avg_anion_electron_affinity,minimum_EN_difference,maximum_EN_difference,...,is_centrosymmetric,n_symmetry_ops,dimensionality,crystal_system_ohe_cubic,crystal_system_ohe_hexagonal,crystal_system_ohe_monoclinic,crystal_system_ohe_orthorhombic,crystal_system_ohe_tetragonal,crystal_system_ohe_triclinic,crystal_system_ohe_trigonal
mp-623511,0.0,0.0,0.0,0.0,1,0.000000,0.000000,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mp-1992,0.0,0.0,0.0,0.0,0,0.190712,0.044698,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mp-27502,0.0,0.0,0.0,0.0,1,0.088829,0.016742,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mp-770629,0.0,0.0,0.0,0.0,1,0.779730,0.171021,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mp-27202,0.0,0.0,0.0,0.0,0,0.070306,0.004246,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-565480,0.0,0.0,0.0,0.0,1,0.820234,0.136348,0.0,0.0,0.0,...,1.0,4.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
mp-773740,0.0,0.0,0.0,0.0,1,0.685139,0.140569,0.0,0.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
mp-25604,0.0,0.0,0.0,0.0,0,0.443019,0.085693,0.0,0.0,0.0,...,1.0,4.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
mp-764227,0.0,0.0,0.0,0.0,1,0.774266,0.185501,0.0,0.0,0.0,...,1.0,4.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
